**Importación y limpieza de datos**

In [1]:
import pandas as pd
import numpy as np
import math
import itertools
import matplotlib.pyplot as plt

In [4]:
dataset=pd.read_csv("/content/titanic.csv")

Exploración de datos con Pandas Profiling

In [5]:
pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

     \ 38.5MB 14.9MB/s
     |████████████████████████████████| 81kB 4.9MB/s 
     |████████████████████████████████| 1.1MB 22.8MB/s 
     |████████████████████████████████| 3.2MB 53.2MB/s 
     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 81kB 6.0MB/s 
     |████████████████████████████████| 296kB 43.6MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.12.0-py2.py3-none-any.whl size=243829 sha256=8424ab5ff413046636ebbee596111f9094bb858552ac7f235e1493d058763cc9
  Stored in directory: /tmp/pip-ephem-wheel-cache-rh6xc42_/wheels/56/c2/dd/8d945b0443c35df7d5f62fa9e9ae105a2d8b286302b92e0109
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp37-none-any.whl size=27085 sha256=a5727baf8b97a28a88c29e06d8ee7f4e142fd82f71207e4cee43f42ffb6fe116
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for phik: filename=phik-0.11.2-cp37-none-any.whl size=1107413 sha256

In [8]:
from pandas_profiling import ProfileReport
profile = ProfileReport(dataset, explorative=True)
profile.to_file("Pandas_profiling_output.html")

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Limpieza de datos

In [12]:
clean_dataset=dataset.drop(['Name','Ticket'],axis=1)


In [14]:
clean_dataset['Age']=clean_dataset['Age'].fillna(math.ceil(clean_dataset['Age'].mean()))


One hot encoding de varibles categóricas

In [15]:
def encode_target(df, target_column):
    df_mod = df.copy()
    targets = df_mod[target_column].unique()
    map_to_int = {name: n for n, name in enumerate(targets)}
    df_mod[target_column] = df_mod[target_column].replace(map_to_int)

    return (df_mod, targets) 

clean_dataset, sex_targets = encode_target(clean_dataset, "Sex")

clean_dataset, embarked_targets = encode_target(clean_dataset, "Embarked")

clean_dataset, cabin_targets = encode_target(clean_dataset, "Cabin")


In [16]:
clean_dataset.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Cabin            int64
Embarked         int64
dtype: object

**Tpot model**

In [17]:
!pip install tpot

     |████████████████████████████████| 92kB 3.8MB/s 
     |████████████████████████████████| 163kB 24.1MB/s 
     |████████████████████████████████| 157.5MB 88kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp37-none-any.whl size=11954 sha256=864514020c9adff63ec46f98d16c4d14ad3d4cb01cfd6017b787ec646df27872
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit
  Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [18]:
from tpot import TPOTClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [22]:
from sklearn.model_selection import train_test_split



In [49]:
X = clean_dataset.loc[:, clean_dataset.columns != 'Survived']
y = clean_dataset.loc[:, clean_dataset.columns == 'Survived']

In [26]:
X.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,3,0,22.0,1,0,7.2500,0,0
1,2,1,1,38.0,1,0,71.2833,1,1
2,3,3,1,26.0,0,0,7.9250,0,0
3,4,1,1,35.0,1,0,53.1000,2,0
4,5,3,0,35.0,0,0,8.0500,0,0


In [27]:
y.head()

,Survived
0,0
1,1
2,1
3,1
4,0


In [29]:

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    train_size=0.75, test_size=0.25, random_state=42)

tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, random_state=42)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_digits_pipeline.py')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8098417686006061

Generation 2 - Current best internal CV score: 0.8233419369318821

Generation 3 - Current best internal CV score: 0.8248681405005049

Generation 4 - Current best internal CV score: 0.8263494557288744

Generation 5 - Current best internal CV score: 0.8338570306362924

Best pipeline: RandomForestClassifier(SGDClassifier(input_matrix, alpha=0.0, eta0=0.01, fit_intercept=True, l1_ratio=0.25, learning_rate=constant, loss=modified_huber, penalty=elasticnet, power_t=0.0), bootstrap=True, criterion=gini, max_features=0.4, min_samples_leaf=3, min_samples_split=5, n_estimators=100)
0.8161434977578476


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**Autokeras**

In [30]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.3MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=19d814a8e01602f5c5e074227e355bb88f02c7ca48ea29b836a0e42318553777
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=e7746b248fb368efcfedd5f572b937f82335991cfcec25f3c5cf7756e5aca388
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [31]:
!pip install autokeras

     |████████████████████████████████| 174kB 12.1MB/s 


In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(668, 9) (223, 9) (668, 1) (223, 1)


In [36]:
from autokeras import StructuredDataRegressor
search = StructuredDataRegressor(max_trials=15, loss='mean_absolute_error')

In [39]:
search.fit(x=X_train, y=y_train, verbose=1)

Trial 16 Complete [00h 00m 08s]
val_loss: 0.17775560915470123

Best val_loss So Far: 0.17775560915470123
Total elapsed time: 00h 08m 07s
INFO:tensorflow:Oracle triggered exit
Epoch 1/36
21/21 [==============================] - 1s 2ms/step - loss: 0.4219 - mean_squared_error: 0.3173
Epoch 2/36
21/21 [==============================] - 0s 2ms/step - loss: 0.2888 - mean_squared_error: 0.1773
Epoch 3/36
21/21 [==============================] - 0s 2ms/step - loss: 0.2711 - mean_squared_error: 0.1741
Epoch 4/36
21/21 [==============================] - 0s 2ms/step - loss: 0.2548 - mean_squared_error: 0.1672
Epoch 5/36
21/21 [==============================] - 0s 2ms/step - loss: 0.2418 - mean_squared_error: 0.1580
Epoch 6/36
21/21 [==============================] - 0s 2ms/step - loss: 0.2435 - mean_squared_error: 0.1647
Epoch 7/36
21/21 [==============================] - 0s 2ms/step - loss: 0.2336 - mean_squared_error: 0.1635
Epoch 8/36
21/21 [==============================] - 0s 2ms/step - los

In [40]:
mae, _ = search.evaluate(X_test, y_test, verbose=0)
print('MAE: %.3f' % mae)

MAE: 0.251


In [41]:
model = search.export_model()

In [42]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 9)]               0         
_________________________________________________________________
multi_category_encoding (Mul (None, 9)                 0         
_________________________________________________________________
normalization (Normalization (None, 9)                 19        
_________________________________________________________________
dense (Dense)                (None, 256)               2560      
_________________________________________________________________
re_lu (ReLU)                 (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0     

**H2O**

In [ ]:
guardar el datsset limpio com csv en colab

In [62]:

clean_dataset.to_csv('/content/clean_dataset.csv', index = False)

In [43]:
! pip install h2o

     |████████████████████████████████| 164.6MB 68kB/s 
  Created wheel for h2o: filename=h2o-3.32.0.4-py2.py3-none-any.whl size=164670979 sha256=f8c7d1f08a4711cd1b6a1931fba4954a475f718e1924f0c4852e88004ed1a643
  Stored in directory: /root/.cache/pip/wheels/65/f4/0d/c9bb958d70c2e015c968cb91cbd7f1b486933056d422337d75
Successfully built h2o


In [44]:
import h2o
from h2o.automl import H2OAutoML

In [45]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpyxf4l5cp
  JVM stdout: /tmp/tmpyxf4l5cp/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpyxf4l5cp/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,1 month and 13 days
H2O_cluster_name:,H2O_from_python_unknownUser_qzu35h
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


importar dataset limpio

In [63]:
df=h2o.import_file('/content/clean_dataset.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [83]:
y="Survived"
x=df.columns
x.remove(y)

In [84]:
splits = df.split_frame(ratios = [0.8], seed = 1)
train = splits[0]
test = splits[1]

In [85]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x,y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [86]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20210315_125709,0.126454,0.355604,0.126454,0.254647,0.249706
StackedEnsemble_AllModels_AutoML_20210315_125709,0.12816,0.357994,0.12816,0.256075,0.251409
DeepLearning_grid__2_AutoML_20210315_125709_model_1,0.129691,0.360127,0.129691,0.264972,0.253093
GBM_grid__1_AutoML_20210315_125709_model_1,0.130363,0.361058,0.130363,0.268698,0.254046
GBM_2_AutoML_20210315_125709,0.130654,0.36146,0.130654,0.259257,0.254026
DRF_1_AutoML_20210315_125709,0.133178,0.364935,0.133178,0.249832,0.256443
GBM_3_AutoML_20210315_125709,0.134359,0.36655,0.134359,0.26199,0.258762
GBM_1_AutoML_20210315_125709,0.136141,0.368973,0.136141,0.259968,0.259301
GBM_4_AutoML_20210315_125709,0.136539,0.369512,0.136539,0.261337,0.259769
DeepLearning_grid__1_AutoML_20210315_125709_model_1,0.138142,0.371674,0.138142,0.261956,0.262204


In [87]:
preds = aml.predict(test)


stackedensemble prediction progress: |████████████████████████████████████| 100%


In [88]:
preds = aml.leader.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [89]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = 'ALL')
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle,training_time_ms,predict_time_per_row_ms
StackedEnsemble_BestOfFamily_AutoML_20210315_125709,0.126454,0.355604,0.126454,0.254647,0.249706,156,0.078147
StackedEnsemble_AllModels_AutoML_20210315_125709,0.12816,0.357994,0.12816,0.256075,0.251409,191,0.063127
DeepLearning_grid__2_AutoML_20210315_125709_model_1,0.129691,0.360127,0.129691,0.264972,0.253093,24122,0.054926
GBM_grid__1_AutoML_20210315_125709_model_1,0.130363,0.361058,0.130363,0.268698,0.254046,87,0.013422
GBM_2_AutoML_20210315_125709,0.130654,0.36146,0.130654,0.259257,0.254026,114,0.012271
DRF_1_AutoML_20210315_125709,0.133178,0.364935,0.133178,0.249832,0.256443,159,0.017129
GBM_3_AutoML_20210315_125709,0.134359,0.36655,0.134359,0.26199,0.258762,173,0.011526
GBM_1_AutoML_20210315_125709,0.136141,0.368973,0.136141,0.259968,0.259301,121,0.008193
GBM_4_AutoML_20210315_125709,0.136539,0.369512,0.136539,0.261337,0.259769,147,0.00704
DeepLearning_grid__1_AutoML_20210315_125709_model_1,0.138142,0.371674,0.138142,0.261956,0.262204,19380,0.004525


In [90]:
preds.head()

predict
0.530118
0.659518
0.310711
0.124329
0.405746
0.120884
0.269198
0.232874
0.192925
0.144377
